# Système de recommandation Pierre

## L'objectif est de créer un système de recommandation de joueurs de Fantasy Premier League (FPL) à partir de leurs performances passées qui sont dans le dossier data du git.

In [6]:
# Import des librairies
import pandas as pd
import numpy as np
import pulp
import matplotlib.pyplot as plt
import seaborn as sns
import os
import unidecode
import glob
import re
import datetime
import warnings
warnings.filterwarnings('ignore')

In [7]:
folderpath = r'../data/input/fantasypremierleague/'

players_1617_df = pd.read_csv(folderpath+'players_raw_1617.csv', encoding='latin')
players_1718_df = pd.read_csv(folderpath+'players_raw_1718.csv',encoding='latin')
players_1819_df = pd.read_csv(folderpath+'players_raw_1819.csv',encoding='latin')
players_1920_df = pd.read_csv(folderpath+'players_raw_1920.csv',encoding='latin')
players_2021_df = pd.read_csv(folderpath+'players_raw_2021.csv',encoding='latin')
players_2122_df = pd.read_csv(folderpath+'players_raw_2122.csv',encoding='latin')
players_2223_df = pd.read_csv(folderpath+'players_raw_2223.csv',encoding='latin')
players_2324_df = pd.read_csv(folderpath+'players_raw_2324.csv',encoding='latin')

gws_1617_df = pd.read_csv(folderpath+'merged_gw_1617.csv',encoding='latin')
gws_1718_df = pd.read_csv(folderpath+'merged_gw_1718.csv',encoding='latin')
gws_1819_df = pd.read_csv(folderpath+'merged_gw_1819.csv',encoding='latin')
gws_1920_df = pd.read_csv(folderpath+'merged_gw_1920.csv',engine='python')
gws_2021_df = pd.read_csv(folderpath+'merged_gw_2021.csv',encoding='latin')
gws_2122_df = pd.read_csv(folderpath+'merged_gw_2122.csv',encoding='latin')
gws_2223_df = pd.read_csv(folderpath+'merged_gw_2223.csv',encoding='latin')
gws_2324_df = pd.read_csv(folderpath+'merged_gw_2324.csv',encoding='latin')

team_codes_df = pd.read_csv('../data/teams.csv', encoding='latin')


In [8]:
team_codes_df.columns.values[2:] = team_codes_df.columns[2:].str.replace('team_', '')

In [9]:
player_df_list = [players_1617_df, players_1718_df, players_1819_df, players_1920_df, players_2021_df, players_2122_df, players_2223_df, players_2324_df]
gw_df_list = [gws_1617_df, gws_1718_df, gws_1819_df, gws_1920_df, gws_2021_df, gws_2122_df, gws_2223_df, gws_2324_df]

In [10]:
# append season and season index to dfs

seasons = ['1617', '1718', '1819', '1920', '2021', '2122', '2223', '2324']
season_nums = list(range(len(seasons)))

for i in range(len(seasons)):

    player_df_list[i]['season'] = seasons[i]
    gw_df_list[i]['season'] = seasons[i]

    player_df_list[i]['season_num'] = season_nums[i]
    gw_df_list[i]['season_num'] = season_nums[i]

In [11]:
# combine dataframes from all seasons into one

players_df = pd.concat(player_df_list)
gws_df = pd.concat(gw_df_list)

players_df.reset_index(inplace=True)
gws_df.reset_index(inplace=True)

In [12]:
## Get full name
# Cleans up accents and also makes processing easier

def get_full_name_playerdf(first_name, second_name):
    full_name = first_name +'_' + second_name
    full_name = full_name.replace(" ", "_")
    full_name = full_name.replace("-", "_")
    full_name = unidecode.unidecode(full_name)

    return full_name

In [13]:
# Translate player positions into string for easier readability

positions_dict = {
    1: 'Keeper',
    2: 'Defender',
    3: 'Midfielder',
    4: 'Forward'
}

players_df['full_name'] = players_df.apply(lambda x: get_full_name_playerdf(x.first_name, x.second_name), axis=1).str.lower()
players_df['position'] = players_df.element_type.map(positions_dict)
players_df['starting_cost'] = players_df.now_cost - players_df.cost_change_start_fall
players_df['cost_bin'] = players_df.now_cost.apply(lambda x: np.floor(x/10))

gws_df['full_name'] = gws_df.name.str.replace('_\d+','')
gws_df['full_name'] = gws_df['full_name'].str.replace(" ", "_").str.replace("-", "_").str.replace('_\d+','')
gws_df['full_name'] = gws_df['full_name'].apply(lambda x: unidecode.unidecode(x))
gws_df['full_name'] = gws_df['full_name'].str.lower()

In [14]:
def clean_gw_df(player_df, gw_df, team_codes_df):

    # Returns a df with player position, player's team name, and opponent's team name

    pdf = player_df.copy()[['full_name', 'season', 'position', 'player_team_name']]
    gdf = gw_df.copy()

    gdf = gdf.merge(pdf, on=['full_name', 'season'], how='left')


    dfs = []
    for s, group in gdf.groupby('season'):

        temp_code_df = team_codes_df[['code', s]]
        temp_code_df = temp_code_df.dropna()

        group = group[['opponent_team']]
        group['opponent_team_name'] = group.opponent_team.map(temp_code_df.set_index(s).team)
        dfs.append(group[['opponent_team_name']])

    out_df = pd.concat(dfs, axis=0)
    out_df = pd.concat([gdf, out_df], axis=1)

    return out_df

In [15]:
def make_available_players_df(this_season_player_df, last_season_player_df):


    last_season_player_df = last_season_player_df[last_season_player_df.minutes > 0]
    last_season_player_df = last_season_player_df[['full_name', "total_points"]]
    last_season_player_df.rename(columns={'total_points': "total_points_last_season"},
                                inplace=True)

    available_players_df = pd.merge(this_season_player_df,
                                    last_season_player_df,
                                   on='full_name', how='left')

    available_players_df.total_points_last_season = available_players_df.groupby(['position', 'cost_bin']).total_points_last_season.transform(lambda x: x.fillna(x.mean()))

    return available_players_df

In [16]:
current_season_player_df = players_df[players_df.season=='2324']
previous_season_player_df = players_df[players_df.season=='2223']

available_players_df = make_available_players_df(current_season_player_df, previous_season_player_df)
available_players_df

,index,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,...,saves_per_90,selected_rank,selected_rank_type,starts,starts_per_90,full_name,position,starting_cost,cost_bin,total_points_last_season
0,0,0,0,0,0.0,0.0,0,232223,0,0,...,0.0,302.0,47.0,0.0,0.00,folarin_balogun,Forward,43,4.0,25.500000
1,1,0,0,0,NaN,NaN,0,58822,0,0,...,0.0,248.0,94.0,0.0,0.00,ca(c)dric_alves_soares,Defender,38,3.0,10.000000
2,2,1,0,12,100.0,100.0,0,153256,0,0,...,0.0,399.0,129.0,0.0,0.00,mohamed_elneny,Midfielder,43,4.0,6.000000
3,3,3,0,69,100.0,100.0,1,438098,0,0,...,0.0,381.0,122.0,2.0,0.85,fa!bio_ferreira_vieira,Midfielder,53,5.0,40.000000
4,4,0,2,132,100.0,100.0,3,226597,0,0,...,0.0,27.0,9.0,7.0,0.96,gabriel_dos_santos_magalhapses,Defender,46,4.0,146.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,728,0,0,27,NaN,NaN,0,87835,0,0,...,0.0,324.0,126.0,1.0,0.40,matt_doherty,Defender,41,4.0,35.000000
729,729,0,0,0,100.0,100.0,0,231480,0,0,...,0.0,591.0,210.0,0.0,0.00,santiago_bueno,Defender,45,4.0,54.375000
730,730,0,0,0,NaN,NaN,0,593001,0,0,...,0.0,666.0,269.0,0.0,0.00,enso_gonza!lez,Midfielder,50,5.0,77.463768
731,731,0,0,0,NaN,NaN,0,531363,0,0,...,0.0,432.0,63.0,0.0,0.00,nathan_fraser,Forward,45,4.0,25.500000


In [17]:
def get_cheapest_players(player_df):

    cheapest_player_names = []
    total_cost = 0

    # for each position, sort the players by cost (in ascending order)
    # then, get the player with the most number of points

    for position, group in player_df.groupby('position'):
        cheapest_players =  group[(group.starting_cost == group.starting_cost.min())]
        top_cheapest_player = cheapest_players[cheapest_players.total_points == cheapest_players.total_points.max()]

        cheapest_player_name = top_cheapest_player.full_name.values[0]

        cheapest_player_names += [cheapest_player_name]
        total_cost += top_cheapest_player.starting_cost.values[0]

        print(position, ": ", cheapest_player_name )

    return cheapest_player_names, total_cost

In [18]:
bench_players, bench_cost = get_cheapest_players(available_players_df)
bench_players

Defender :  george_baldock
Forward :  william_osula
Keeper :  thomas_strakosha
Midfielder :  elliot_anderson


['george_baldock', 'william_osula', 'thomas_strakosha', 'elliot_anderson']

In [19]:
def make_decision_variables(player_df):
    return [pulp.LpVariable(i, cat="Binary") for i in player_df.full_name]

In [20]:
def make_optimization_function(player_df, decision_variables):
    op_func = ""

    for i, player in enumerate(decision_variables):
        op_func += player_df.total_points_last_season[i]*player

    return op_func

In [21]:
def make_cash_constraint(player_df, decision_variables, available_cash):
    total_paid = ""
    for rownum, row in player_df.iterrows():
        for i, player in enumerate(decision_variables):
            if rownum == i:
                formula = row['starting_cost']*player
                total_paid += formula

    return (total_paid <= available_cash)

In [22]:
def make_player_constraint(position, n, decision_variables, player_df):

    total_n = ""

    player_positions = player_df.position

    for i, player in enumerate(decision_variables):
        if player_positions[i] == position:
            total_n += 1*player

    return(total_n == n)

In [23]:
def add_team_constraint(prob, player_df, decision_variables):

    for team, group in player_df.groupby('team_code'):
        team_total = ''

        for player in decision_variables:
            if player.name in group.full_name.values:
                formula = 1*player
                team_total += formula


        prob += (team_total <= 3)

In [24]:
available_cash = 1000 - bench_cost

prob = pulp.LpProblem('InitialTeam', pulp.LpMaximize)

decision_variables = make_decision_variables(available_players_df)
prob += make_optimization_function(available_players_df, decision_variables)
prob += make_cash_constraint(available_players_df, decision_variables, available_cash)
prob += make_player_constraint("Keeper", 1, decision_variables, available_players_df)
prob += make_player_constraint("Defender", 4, decision_variables, available_players_df)
prob += make_player_constraint("Midfielder", 4, decision_variables, available_players_df)
prob += make_player_constraint("Forward", 2, decision_variables, available_players_df)

add_team_constraint(prob, available_players_df, decision_variables)

In [25]:
## Solve

prob.writeLP('InitialTeam.lp')
optimization_result = prob.solve()

In [26]:
## Get initial team

def get_initial_team(prob, player_df):

    variable_names = [v.name for v in prob.variables()]
    variable_values = [v.varValue for v in prob.variables()]

    initial_team = pd.merge(pd.DataFrame({'full_name': variable_names,
                  'selected': variable_values}),
                                       player_df, on="full_name")

    initial_team = initial_team[initial_team.selected==1.0]

    return initial_team

In [27]:
initial_team_df = get_initial_team(prob, available_players_df)
initial_team_df[['full_name', "position", "starting_cost"]]

,full_name,position,starting_cost
85,ben_mee,Defender,46
105,bruno_borges_fernandes,Midfielder,81
168,david_raya_martin,Keeper,46
218,erling_haaland,Forward,140
227,fabian_scha$?r,Defender,54
244,gabriel_martinelli_silva,Midfielder,74
270,harry_kane,Forward,125
419,kieran_trippier,Defender,75
481,marcus_rashford,Midfielder,82
488,martin_adegaard,Midfielder,83


In [28]:
## Sanity check

def sanity_check(team_df):
    print('Sanity check for starting 11: ')
    print('*'*88)

    print('Number of players in each position: ')
    for pos, group in team_df.groupby('position'):
        print(pos, ': ', len(group), sep='')


    print('*'*88)
    print('Number of players from each team: ')
    print(team_df.position.count())

    print('*'*88)
    print('Total cost:', team_df.starting_cost.sum())

In [29]:
sanity_check(initial_team_df)

Sanity check for starting 11: 
****************************************************************************************
Number of players in each position: 
Defender: 4
Forward: 2
Keeper: 1
Midfielder: 4
****************************************************************************************
Number of players from each team: 
11
****************************************************************************************
Total cost: 843


In [30]:
captain = get_initial_team(prob, previous_season_player_df).sort_values("total_points", ascending=False).head(1).full_name.values[0]

total_points = current_season_player_df[current_season_player_df.full_name.isin(initial_team_df.full_name)].total_points.sum()
total_points += current_season_player_df[current_season_player_df.full_name==captain].total_points

print("Total points for 23/24 season:", total_points.values[0])

Total points for 23/24 season: 468


In [31]:
def get_team_points(was_home, h_score, a_score):

    if h_score == a_score:
        return 1

    if h_score > a_score:
        if was_home:
            return 3
        else:
            return 0

    if h_score < a_score:
        if was_home:
            return 0
        else:
            return 3

In [32]:
def get_opponent_points(team_points):
    if team_points == 1:
        return 1

    if team_points == 3:
        return 0

    if team_points == 0:
        return 3

In [33]:
gws_df['team_points']= gws_df.apply(lambda x: get_team_points(x.was_home, x.team_h_score, x.team_a_score), axis=1)
gws_df['opponent_points'] = gws_df.team_points.apply(lambda x: get_opponent_points(x))

In [38]:
def player_lag_features(gw_df, features, lags):

    out_df = gw_df.copy()
    lagged_features = []

    for feature in features:

        for lag in lags:

            lagged_feature = 'last_' + str(lag) + '_' + feature

            if lag == 'all':
                out_df[lagged_feature] = out_df.sort_values('round').groupby(['season', 'full_name'])[feature]\
            .apply(lambda x: x.cumsum() - x)

            else:

                out_df[lagged_feature] = out_df.sort_values('round').groupby(['season', 'full_name'])[feature]\
                .apply(lambda x: x.rolling(min_periods=1, window=lag+1).sum() - x)

            lagged_features.append(lagged_feature)

    return out_df, lagged_features

In [39]:
def team_lag_features(gw_df, features, lags):
    out_df = gw_df.copy()
    lagged_features = []

    for feature in features:

        ## Create a df for each feature
        ## Then, self-join so that the opponent info for that feature is included
        ## Then, create lagged features and join the columns to the feature df
        ## Do the same for the opponent feature
        ## Exit loop, merge with the original df

        feature_name = feature + '_team'
        opponent_feature_name = feature_name + '_opponent'


        feature_team = out_df.groupby(['player_team_name', 'season', 'round', 'kickoff_time', 'opponent_team_name'])\
                        [feature].max().rename(feature_name).reset_index()

        # self join to get opponent info

        feature_team = feature_team.merge(feature_team,
                          left_on=['player_team_name', 'season', 'round', 'kickoff_time', 'opponent_team_name'],
                          right_on=['opponent_team_name', 'season', 'round', 'kickoff_time', 'player_team_name'],
                          how='left',
                          suffixes=('', '_opponent'))




        for lag in lags:
            lagged_feature_name = 'last_' + str(lag) + '_' + feature_name
            lagged_opponent_feature_name = 'opponent_last_' + str(lag) + '_' + feature


            if lag == 'all':

                feature_team[lagged_feature_name] = feature_team.sort_values('round').groupby('player_team_name')[feature_name]\
                                                .apply(lambda x: x.cumsum() - x)

                feature_team[lagged_opponent_feature_name] = feature_team.groupby('player_team_name')[opponent_feature_name]\
                                                .apply(lambda x: x.cumsum() - x)
            else:


                feature_team[lagged_feature_name] = feature_team.sort_values('round').groupby('player_team_name')[feature_name]\
                                                    .apply(lambda x: x.rolling(min_periods=1,
                                                                              window=lag+1).sum()-x)

                feature_team[lagged_opponent_feature_name] = feature_team.groupby('player_team_name')[opponent_feature_name]\
                                                    .apply(lambda x: x.rolling(min_periods=1,
                                                                              window=lag+1).sum()-x)

            lagged_features.extend([lagged_feature_name, lagged_opponent_feature_name])

        out_df = out_df.merge(feature_team,
                             on=['player_team_name', 'season', 'round', 'kickoff_time', 'opponent_team_name'],
                             how='left')


        return out_df, lagged_features

In [40]:
player_features_to_lag = [
    'assists',
     'bonus',
     'bps',
     'creativity',
     'clean_sheets',
     'goals_conceded',
     'goals_scored',
     'ict_index',
     'influence',
     'minutes',
     'threat']

team_features_to_lag = ['goals_conceded', 'goals_scored', 'team_points', 'opponent_points']

In [41]:
lagged_gw_df_players, lagged_player_features = player_lag_features(gws_df, player_features_to_lag, ['all', 1, 3, 5])

TypeError: incompatible index of inserted column with frame index

In [ ]:
lagged_gw_df, lagged_team_features = team_lag_features(lagged_gw_df_players, team_features_to_lag, ['all', 1, 3, 5])